In [ ]:
import requests

In [ ]:
DATADIR = '/data/uscuni-ulce/cadastre_buildings_raw/'

### Download german building footprints

In [ ]:
import geopandas as gpd
from io import BytesIO
import re

def process_wfs(region_name, size, url, params, count, start=0,):
    params = params.copy()
    i = start
    for i in range(start, size+count, count):
        
        print(i)
        params['count'] = count,
        params['startIndex'] = i,

        # Make the request
        response = requests.get(url, params=params)
        
        if response.status_code != 200:
            print(i, count)
            print(response.content)
            break

        try:
            gdf = gpd.read_file(BytesIO(response.content))
        except Exception as e:
            print(e)
            break
            
        gdf.to_parquet(DATADIR + f"buildings_{region_name}_{i}.pq")

In [ ]:
def get_size(url, params):
    params = params.copy()
    params['resultType'] = 'hits'
    response = requests.get(url, params=params)
    if response.status_code == 200:
        size = re.findall(r'numberMatched="[0-9]+"', str(response.content))
        size = int(size[0].split('=')[-1][1:-1])
        return size

    return 'unknown'

In [ ]:
%%time

# BB
url = "https://isk.geobasis-bb.de/ows/alkis_vereinf_wfs"

# Parameters for the GetFeature request

params = {
        "SERVICE": "WFS",
        "REQUEST": "GetFeature",
        "TYPENAMES": "ave:GebaeudeBauwerk",
        "SRSNAME": "EPSG:4326",
        "VERSION": '2.0.0',
    }

size = get_size(url, params)
print(size)


In [ ]:
count = 100_000
process_wfs("germany_brandeburg", size, url, params, count)

In [ ]:
#hessen
params = {
        "SERVICE": "WFS",
        "REQUEST": "GetFeature",
        "TYPENAMES": "bu-core2d:Building",
        "SRSNAME": "EPSG:4326",
        "VERSION": '2.0.0',
    }
url = 'https://inspire-hessen.de/ows/services/org.2.29e99e26-ab08-474a-92c4-9e8f6bc679ea_wfs?'
size = get_size(url, params)
print(size)


In [ ]:

count = 100000
process_wfs("germany_hessen", size, url, params, count, start=3900000)

In [ ]:
#thuringen
i = 0
params = {
        "SERVICE": "WFS",
        "REQUEST": "GetFeature",
        "TYPENAMES": "ave:GebaeudeBauwerk",
        "SRSNAME": "EPSG:5650",
        "VERSION": '2.0.0',
    }

url = ' https://www.geoproxy.geoportal-th.de/geoproxy/services/adv_alkis_wfs'
size = get_size(url, params)
print(size)

In [ ]:
count = 100_000
process_wfs("germany_th", size, url, params, count)

In [ ]:
#sachsen
i = 0
params = {
        "SERVICE": "WFS",
        "REQUEST": "GetFeature",
        "TYPENAMES": "ave:GebaeudeBauwerk",
        "SRSNAME": "EPSG:25833",
        "VERSION": '2.0.0',
    }
url = 'https://geodienste.sachsen.de/aaa/public_alkis/vereinf/wfs'

size = get_size(url, params)
print(size)

In [ ]:
count = 10000
process_wfs("germany_sachsen", size, url, params, count)

In [ ]:
#sachsen-anhalt
i = 0
params = {
        "SERVICE": "WFS",
        "REQUEST": "GetFeature",
        "TYPENAMES": "bu-core2d:Building",
        "SRSNAME": "EPSG:25833",
        "VERSION": '2.0.0',
    }
url = 'https://www.geodatenportal.sachsen-anhalt.de/wss/service/INSPIRE_LVermGeo_WFS_ALKIS_BU/guest'
size = get_size(url, params)
print(size)


In [ ]:

count = 10_000
process_wfs("germany_sachsen-anhalt", size, url, params, count)

In [ ]:
#baddwen_wurteburd
params = {
        "SERVICE": "WFS",
        "REQUEST": "GetFeature",
        "TYPENAMES": "bu-core2d:Building",
        "SRSNAME": "EPSG:25833",
        "VERSION": '2.0.0',
    }
url = 'https://owsproxy.lgl-bw.de/owsproxy/wfs/WFS_INSP_BW_Gebaeude_ALKIS'

size = get_size(url, params)
print(size)

In [ ]:

count = 10000
process_wfs("germany_bw", size, url, params, count)

In [ ]:
#movmovi - vavdfav
i = 0
params = {
        "SERVICE": "WFS",
        "REQUEST": "GetFeature",
        "TYPENAMES": "ave:GebaeudeBauwerk",
        "SRSNAME": "EPSG:5650",
        "VERSION": '2.0.0',
    }
url = 'https://www.geodaten-mv.de/dienste/alkis_wfs_einfach'
size = get_size(url, params)
print(size)


In [ ]:
count = 10000
process_wfs("germany_mv", size, url, params, count)

In [ ]:
### RLP

# # https://www.geoportal.rlp.de/mapbender/php/mod_inspireDownloadFeed.php?id=30cc0a9a-cbad-66e3-68a5-7b75544814d8&type=DATASET&generateFrom=wfs&wfsid=519&featuretypeid=2713
# #rlp
# i = 0
# params = {
#         "SERVICE": "WFS",
#         "REQUEST": "GetFeature",
#         "TYPENAMES": "bu-core2d:Building",
#         "SRSNAME": "EPSG:25832",
#         "VERSION": '2.0.0',
#     }
# url = 'https://www.geoportal.rlp.de/registry/wfs/519'
# size = get_size(url, params)
# print(size)


# # count = 100000
# # process_wfs("germany_rlp", url, params, count)


for i in range(0, 3320444, 4000):
    url = f'https://www.geoportal.rlp.de/spatial-objects/519/collections/ave:GebaeudeBauwerk/items?offset={i}&limit=4000&f=json'
    gdf = gpd.read_file(url)
    gdf.to_parquet(DATADIR + f'buildings_germany_rlp_{i}.pq')
    print(i)

In [ ]:

# # #saarland
# # i = 0
# # params = {
# #         "SERVICE": "WFS",
# #         "REQUEST": "GetFeature",
# #         "TYPENAMES": "bu-core2d:Building",
# #         "SRSNAME": "EPSG:4326",
# #         "VERSION": '2.0.0',
# #     }
# # url = 'https://geoportal.saarland.de/gdi-sl/inspirewfs_2D_Gebaeude_ALKIS'
# # size = get_size(url, params)
# # print(size)

# # params['count'] = size
# # response = requests.get(url, params=params)


for i in range(0, 622841, 500):
    url = f'https://geoportal.saarland.de/spatial-objects/406/collections/bu-core2d:Building/items?offset={i}&limit=500&f=json'
    gdf = gpd.read_file(url)
    gdf.to_parquet(DATADIR + f'buildings_germany_saarland_{i}.pq')
    print(i)

In [ ]:
# #nrw
# i = 0
# params = {
#         "SERVICE": "WFS",
#         "REQUEST": "GetFeature",
#         "TYPENAMES": "bu-core2d:Building",
#         "SRSNAME": "EPSG:4326",
#         "VERSION": '2.0.0',
#     }
# url = 'https://www.wfs.nrw.de/geobasis/wfs_nw_inspire-gebaeude-2d_alkis'

# size = get_size(url, params)
# size
# count = 10000
# process_wfs("germany_nrw", url, params, count, start=)

step = 5_000

for i in range(0, 12646244, step):
    url = f'https://ogc-api.nrw.de/lika/v1/collections/gebaeude_bauwerk/items?offset={i}&limit={step}&f=json'
    gdf = gpd.read_file(url)
    gdf.to_parquet(DATADIR + f'buildings_germany_nrw_{i}.pq')
    print(i, gdf.shape)

In [ ]:
## NI

In [ ]:
# https://ni-lgln-opengeodata.hub.arcgis.com/apps/1cadea38ffb341bc83019b3d7f787dd3/explore


# #niedersachsen
# i = 0
# params = {
#         "SERVICE": "WFS",
#         "REQUEST": "GetFeature",
#         "TYPENAMES": "bu-core2d:Building",
#         "VERSION": '2.0.0',
#     }
# url = 'https://www.inspire.niedersachsen.de/doorman/noauth/alkis-dls-bu-core2d?'

# size = get_size(url, params)
# print(size)

# count = 10000
# process_wfs("germany_niedersachsen", url, params, count)

In [ ]:
links = gpd.read_file('https://arcgis-geojson.s3.eu-de.cloud-object-storage.appdomain.cloud/alkis-vektor/lgln-opengeodata-alkis-vektor.geojson')

In [ ]:
import urllib.parse
for link in links.zip.values:
    link = urllib.parse.quote(link, safe=':/')
    print(link)
    gdf = gpd.read_file(link, layer='gebaeude')
    gdf.to_parquet(DATADIR + f'buildings_germany_ni_{link.split('/')[-1][:-9]}.pq')

In [ ]:
# ## schlesig hollstein
# params = {
#         "SERVICE": "WFS",
#         "REQUEST": "GetFeature",
#         "TYPENAMES": "bu-core2d:Building",
#         "SRSNAME": "EPSG:4326",
#         "VERSION": '2.0.0',
#     }
# url = 'https://service.gdi-sh.de/SH_INSPIREDOWNLOAD_AIII_BU?service=WFS&version=2.0.0'

# size = get_size(url, params)
# print(size)



# count = 10_000
# process_wfs("germany_sh", url, params, count)

In [ ]:
!wget https://service.gdi-sh.de/SH_OpenGBD/feeds/Atom_SH_ALKIS_vereinf_OpenGBD/Atom_SH_ALKIS_vereinf_OpenGBD.xml

In [ ]:
import xml.etree.ElementTree as ET

# Load the XML file
tree = ET.parse('Atom_SH_ALKIS_vereinf_OpenGBD.xml')
root = tree.getroot()


# Define the namespaces
namespaces = {
    'atom': 'http://www.w3.org/2005/Atom',
    'georss': 'http://www.georss.org/georss',
    'inspire_dls': 'http://inspire.ec.europa.eu/schemas/inspire_dls/1.0',
    'lang': 'ger'
}

# Find all elements with the tag 'entry'
entries = root.findall('atom:entry', namespaces)
codes = []

# Extract and print the 'inspire_dls:spatial_dataset_identifier_code' attribute for each entry
for entry in entries:
    # Find the 'inspire_dls:spatial_dataset_identifier_code' element within the entry
    spatial_code = entry.find('inspire_dls:spatial_dataset_identifier_code', namespaces)
    codes.append(spatial_code.text)

In [ ]:
len(codes)

In [ ]:
for code in codes:
    url = f'https://dienste.gdi-sh.de/WFS_SH_ALKIS_vereinf_OpenGBD?Request=GetFeature&Service=WFS&Version=2.0.0&gemeindeschluessel={code}&StoredQuery_ID=GetGebByGemeinde'
    gdf = gpd.read_file(url, driver='GML')
    gdf.to_parquet(DATADIR + f'buildings_germany_sg_{code}.pq')

In [ ]:
!rm Atom_SH_ALKIS_vereinf_OpenGBD.xml

#### German City regions

In [ ]:


i = 0
params = {
        "SERVICE": "WFS",
        "REQUEST": "GetFeature",
        "TYPENAMES": "bu2d_alkis:Building",
        "SRSNAME": "EPSG:25832",
        "VERSION": '2.0.0',
    }
url = 'https://gdi.berlin.de/services/wms/bu2d_alkis'

count = 100000
process_wfs("germany_berlin", url, params, count)

In [ ]:
import requests 
def download_url(url, save_path, chunk_size=256):
    r = requests.get(url, stream=True, allow_redirects=True)
    print(r.content)
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)

In [ ]:
# bavaria - https://geodaten.bayern.de/opengeodata/OpenDataDetail.html?pn=hausumringe

links = """https://geodaten.bayern.de/odd/m/3/daten/hausumringe/bezirk/data/096_Unterfranken_Hausumringe.zip
https://geodaten.bayern.de/odd/m/3/daten/hausumringe/bezirk/data/094_Oberfranken_Hausumringe.zip
https://geodaten.bayern.de/odd/m/3/daten/hausumringe/bezirk/data/095_Mittelfranken_Hausumringe.zip
https://geodaten.bayern.de/odd/m/3/daten/hausumringe/bezirk/data/093_Oberpfalz_Hausumringe.zip
https://geodaten.bayern.de/odd/m/3/daten/hausumringe/bezirk/data/097_Schwaben_Hausumringe.zip
https://geodaten.bayern.de/odd/m/3/daten/hausumringe/bezirk/data/091_Oberbayern_Hausumringe.zip
https://geodaten.bayern.de/odd/m/3/daten/hausumringe/bezirk/data/092_Niederbayern_Hausumringe.zip"""

In [ ]:
!wget https://geodaten.bayern.de/odd/m/3/daten/hausumringe/bezirk/data/096_Unterfranken_Hausumringe.zip
!wget https://geodaten.bayern.de/odd/m/3/daten/hausumringe/bezirk/data/094_Oberfranken_Hausumringe.zip
!wget https://geodaten.bayern.de/odd/m/3/daten/hausumringe/bezirk/data/095_Mittelfranken_Hausumringe.zip
!wget https://geodaten.bayern.de/odd/m/3/daten/hausumringe/bezirk/data/093_Oberpfalz_Hausumringe.zip
!wget https://geodaten.bayern.de/odd/m/3/daten/hausumringe/bezirk/data/097_Schwaben_Hausumringe.zip
!wget https://geodaten.bayern.de/odd/m/3/daten/hausumringe/bezirk/data/091_Oberbayern_Hausumringe.zip
!wget https://geodaten.bayern.de/odd/m/3/daten/hausumringe/bezirk/data/092_Niederbayern_Hausumringe.zip

In [ ]:
%%time
for i, dl_link in enumerate(links.split('\n')):
    region_name = dl_link.split('_')[-2].lower()
    region_file = dl_link.split('/')[-1]
    gpd.read_file(region_file).to_parquet(DATADIR + f'buildings_germany_bavaria_{region_name}.pq')

In [ ]:
!rm *_Hausumringe.zip

In [ ]:


# #rlp
# i = 0
# params = {
#         "SERVICE": "WFS",
#         "REQUEST": "GetFeature",
#         "TYPENAMES": "bu-core2d:Building",
#         # "SRSNAME": "EPSG:4326",
#         "VERSION": '2.0.0',
#         # "count": 10000,
#         # "startIndex": i
#     }
# url = 'https://geodienste.hamburg.de/HH_WFS_INSPIRE_Gebaeude_2D_ALKIS'

# response = requests.get(url, params=params)

# gdf = gpd.read_file(response.content)
# gdf

In [ ]:


hamburg_url = 'https://daten-hamburg.de/inspire/hh_inspire_gebaeude_2d_alkis/INSPIRE_HH_Gebaeude_2D_ALKIS_2020-07-15.zip'

In [ ]:
!wget https://daten-hamburg.de/inspire/hh_inspire_gebaeude_2d_alkis/INSPIRE_HH_Gebaeude_2D_ALKIS_2020-07-15.zip

In [ ]:
!unzip INSPIRE_HH_Gebaeude_2D_ALKIS_2020-07-15.zip

In [ ]:
gpd.list_layers('result.gml')

In [ ]:
gpd.read_file('result.gml', layer='building').to_parquet(DATADIR + 'buildings_germany_hamburg.pq')

In [ ]:
## cleanup
!rm result.gml
!rm result.gfs
!rm wfs_bu_alkis_2020-07-15.xsd 
!rm INSPIRE_HH_Gebaeude_2D_ALKIS_2020-07-15.zip

In [ ]:
bremen = 'https://gdi2.geo.bremen.de/inspire/download/ADV-Shape/data/ALKIS_AdV_SHP_2024_04_HB.zip'

In [ ]:
!wget https://gdi2.geo.bremen.de/inspire/download/ADV-Shape/data/ALKIS_AdV_SHP_2024_04_HB.zip

In [ ]:
!unzip ALKIS_AdV_SHP_2024_04_HB.zip

In [ ]:
gpd.read_file('Bremen/GebaeudeBauwerk.shp').to_parquet(DATADIR + 'buildings_germany_bremen.pq')

In [ ]:
!rm -r Bremen/

In [ ]:
!rm -r ALKIS_AdV_SHP_2024_04_HB.zip

### Download Austria

In [ ]:
austria = 'https://data.bev.gv.at/download/DLM/DLM_20230125/DLM_8000_BAUWERK_20230125.gpkg'

In [ ]:
%%time
download_url(austria, '../data/DLM_8000_BAUWERK_20230125.gpkg')

In [ ]:
gpd.list_layers( '../data/DLM_8000_BAUWERK_20230125.gpkg')

In [ ]:
import pyogrio

In [ ]:
pyogrio.read_info('../data/DLM_8000_BAUWERK_20230125.gpkg', layer='BWK_8100_BAUWERK_F')

In [ ]:
gpd.read_file('../data/DLM_8000_BAUWERK_20230125.gpkg', layer='BWK_8100_BAUWERK_F').to_parquet(DATADIR + 'buildings_austria.pq')

In [ ]:
# !rm ../data/DLM_8000_BAUWERK_20230125.gpkg

### Download Poland

In [ ]:
# https://mapy.geoportal.gov.pl/iMapLite/atomExplorer/

In [ ]:

links  = """http://mapy.geoportal.gov.pl/wss/service/ATOM/httpauth/download/?fileId=34ad9f34a0b7dc8205a933bb058e91c1&name=podkarpackie_bu.zip
http://mapy.geoportal.gov.pl/wss/service/ATOM/httpauth/download/?fileId=78896afe993bcb34365fe7b8a7f9966f&name=pomorskie_bu.zip
http://mapy.geoportal.gov.pl/wss/service/ATOM/httpauth/download/?fileId=be4bfa1478a6eeedd67faffea96eafda&name=opolskie_bu.zip
http://mapy.geoportal.gov.pl/wss/service/ATOM/httpauth/download/?fileId=c7881c0de4e3d86548ba0e97c50b490a&name=lodzkie_bu.zip
http://mapy.geoportal.gov.pl/wss/service/ATOM/httpauth/download/?fileId=79dcc64d4c0c4e8873365f04c55a4940&name=lubuskie_bu.zip
http://mapy.geoportal.gov.pl/wss/service/ATOM/httpauth/download/?fileId=ac47d220ce942cc3668f703fe4bd7a60&name=mazowieckie_bu.zip
http://mapy.geoportal.gov.pl/wss/service/ATOM/httpauth/download/?fileId=9e87962dd94adddb2838f6321778f04a&name=kujawskopomorskie_bu.zip
http://mapy.geoportal.gov.pl/wss/service/ATOM/httpauth/download/?fileId=8818a617772a6c9dad222dd52acecc29&name=slaskie_bu.zip
http://mapy.geoportal.gov.pl/wss/service/ATOM/httpauth/download/?fileId=4b636d1174ceff606fb1c9c95c145915&name=dolnoslaskie_bu.zip
http://mapy.geoportal.gov.pl/wss/service/ATOM/httpauth/download/?fileId=99c8bdcfc3173a3a7c2d18bf675f4315&name=malopolskie_bu.zip
http://mapy.geoportal.gov.pl/wss/service/ATOM/httpauth/download/?fileId=cb81449840b96ff21fba1a42e094c074&name=lubelskie_bu.zip
http://mapy.geoportal.gov.pl/wss/service/ATOM/httpauth/download/?fileId=9bb1518bbf0d8e22ec338c80dcc83f61&name=zachodniopomorskie_bu.zip
http://mapy.geoportal.gov.pl/wss/service/ATOM/httpauth/download/?fileId=0217fcdf1da066af11a62067530786a9&name=warminskomazurskie_bu.zip
http://mapy.geoportal.gov.pl/wss/service/ATOM/httpauth/download/?fileId=67eb749e1ae54cb7aa9a8aa1d90d8dd7&name=wielkopolskie_bu.zip
http://mapy.geoportal.gov.pl/wss/service/ATOM/httpauth/download/?fileId=fc9fe6a16296acf6a4072a08f5be1342&name=podlaskie_bu.zip
http://mapy.geoportal.gov.pl/wss/service/ATOM/httpauth/download/?fileId=e968c6db383a80a34b702866ede92cb1&name=swietorzyskie_bu.zip"""

In [ ]:
# !wget http://mapy.geoportal.gov.pl/wss/service/ATOM/httpauth/download/?fileId=34ad9f34a0b7dc8205a933bb058e91c1&name=podkarpackie_bu.zip

# !wget http://mapy.geoportal.gov.pl/wss/service/ATOM/httpauth/download/?fileId=78896afe993bcb34365fe7b8a7f9966f&name=pomorskie_bu.zip

# !wget http://mapy.geoportal.gov.pl/wss/service/ATOM/httpauth/download/?fileId=be4bfa1478a6eeedd67faffea96eafda&name=opolskie_bu.zip

# !wget http://mapy.geoportal.gov.pl/wss/service/ATOM/httpauth/download/?fileId=c7881c0de4e3d86548ba0e97c50b490a&name=lodzkie_bu.zip

# !wget http://mapy.geoportal.gov.pl/wss/service/ATOM/httpauth/download/?fileId=79dcc64d4c0c4e8873365f04c55a4940&name=lubuskie_bu.zip

# !wget http://mapy.geoportal.gov.pl/wss/service/ATOM/httpauth/download/?fileId=ac47d220ce942cc3668f703fe4bd7a60&name=mazowieckie_bu.zip

# !wget http://mapy.geoportal.gov.pl/wss/service/ATOM/httpauth/download/?fileId=9e87962dd94adddb2838f6321778f04a&name=kujawskopomorskie_bu.zip

# !wget http://mapy.geoportal.gov.pl/wss/service/ATOM/httpauth/download/?fileId=8818a617772a6c9dad222dd52acecc29&name=slaskie_bu.zip

# !wget http://mapy.geoportal.gov.pl/wss/service/ATOM/httpauth/download/?fileId=4b636d1174ceff606fb1c9c95c145915&name=dolnoslaskie_bu.zip

# !wget http://mapy.geoportal.gov.pl/wss/service/ATOM/httpauth/download/?fileId=99c8bdcfc3173a3a7c2d18bf675f4315&name=malopolskie_bu.zip

# !wget http://mapy.geoportal.gov.pl/wss/service/ATOM/httpauth/download/?fileId=cb81449840b96ff21fba1a42e094c074&name=lubelskie_bu.zip

# !wget http://mapy.geoportal.gov.pl/wss/service/ATOM/httpauth/download/?fileId=9bb1518bbf0d8e22ec338c80dcc83f61&name=zachodniopomorskie_bu.zip

# !wget http://mapy.geoportal.gov.pl/wss/service/ATOM/httpauth/download/?fileId=0217fcdf1da066af11a62067530786a9&name=warminskomazurskie_bu.zip

# !wget http://mapy.geoportal.gov.pl/wss/service/ATOM/httpauth/download/?fileId=67eb749e1ae54cb7aa9a8aa1d90d8dd7&name=wielkopolskie_bu.zip

# !wget http://mapy.geoportal.gov.pl/wss/service/ATOM/httpauth/download/?fileId=fc9fe6a16296acf6a4072a08f5be1342&name=podlaskie_bu.zip

# !wget http://mapy.geoportal.gov.pl/wss/service/ATOM/httpauth/download/?fileId=e968c6db383a80a34b702866ede92cb1&name=swietorzyskie_bu.zip

In [ ]:
links = links.split('\n')

In [ ]:
# https://mapy.geoportal.gov.pl/iMapLite/atomExplorer/

In [ ]:

for i, dl_link in enumerate(links):
    dl_name = dl_link.split('=')[-1]
    download_url(dl_link, dl_name)
    print(dl_name)

In [ ]:
for i, dl_link in enumerate(links[5:]):
    dl_name = dl_link.split('=')[-1]
    if dl_name == 'mazowieckie_bu.zip':
        continue
    gdf = gpd.read_file(dl_name)
    gdf.to_parquet(DATADIR + f'buildings_poland_{dl_name[:-4]}.pq')

In [ ]:
!unzip mazowieckie_bu.zip

In [ ]:
gpd.list_layers('mazowieckie_bu1.gml')

In [ ]:
gpd.list_layers('mazowieckie_bu2.gml')

In [ ]:
gdf = gpd.read_file('mazowieckie_bu1.gml')
gdf.to_parquet(DATADIR + f'buildings_poland_mazowieckie_bu1.pq')

gdf = gpd.read_file('mazowieckie_bu2.gml')
gdf.to_parquet(DATADIR + f'buildings_poland_mazowieckie_bu2.pq')

In [ ]:
!rm mazowieckie_bu1.gml
!rm mazowieckie_bu2.gml

### Download Czechia

In [ ]:
import httpx
url_adm = "https://services.cuzk.cz/gml/inspire/au/epsg-4258/1.zip"
units = pyogrio.read_dataframe(url_adm, layer="AdministrativeUnit")
obce = units[units.LocalisedCharacterString == "Obec"]

In [ ]:
for code in obce["nationalCode"]:
    url_building = f"https://services.cuzk.cz/gml/inspire/bu/epsg-4258/{code}.zip"
    r = httpx.get(url_building, verify=False)

    gdf = gpd.read_file(BytesIO(r.content), layer='Building')
    gdf.to_parquet(DATADIR + f'buildings_czechia_{code}.pq')
    print(code)

### Download Slovakia

In [ ]:
%%time
import httpx
url = "https://opendata.skgeodesy.sk/static/INSPIRE/Buildings/inspire_bu_gpkg.zip"
r = httpx.get(url, verify=False)

In [ ]:
with open("inspire_bu_gpkg.zip", "wb") as f:
    f.write(r.content)

In [ ]:
!unzip inspire_bu_gpkg.zip

In [ ]:
gpd.list_layers('INSPIRE_BU.gpkg')

In [ ]:
gdf = gpd.read_file('INSPIRE_BU.gpkg')
gdf.to_parquet(DATADIR + f'buildings_slovakia_{code}.pq')

In [ ]:
!rm inspire_bu_gpkg.zip
!rm INSPIRE_BU.gpkg 
!rm _license_terms.pdf